# Importar Lib's

In [1]:
import os
import openpyxl
import pandas as pd
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from datetime import datetime
from ipywidgets import interact

# Passo 1: Listar arquivos PDF em uma pasta específica

## Pasta contendo os arquivos PDF

In [2]:
diretorio_lote = r'C:\Users\helym\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\01-HOPPECKE\000_Relatórios de Campo\001_RELATÓRIOS_ENVIADOS\002_PREVENTIVAS\LOTE\COM_NºOBRA'
arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.csv'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

duplicadas = pd.DataFrame(arquivos_pdf, columns=['NUMOS'])
duplicadas['NUMOS'] = duplicadas['NUMOS'].apply(lambda x: x[-22:-10])
lista_arquivos_pdf = duplicadas#lista_arquivos_pdf = duplicadas.astype(int)
lista_arquivos_pdf_duplicada = lista_arquivos_pdf.duplicated().sum()
duplicadas.count()

NUMOS    64
dtype: int64

# Lista de arquivos PDF na pasta

In [3]:
def encontrar_duplicados(duplicadas):
    duplicados = duplicadas[duplicadas.duplicated(subset='NUMOS')]
    return duplicados['NUMOS']

# Exemplo de uso
duplicados = encontrar_duplicados(duplicadas)
print(duplicados)

Series([], Name: NUMOS, dtype: object)


# Passo 2: Ler o arquivo Excel com a lista de arquivos

In [4]:
df_filtered_global_oem = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\helym\projetos_python\dashboard\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global_oem
    df_filtered_global_oem = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DATACONCLUSAO"] >= data_inicial) &
        (df["DATACONCLUSAO"] <= data_final)
    ]
    return df_filtered_global_oem

if __name__ == "__main__":
    df = ler_planilha()

    df["DATACONCLUSAO"] = pd.to_datetime(df["DATACONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = sorted(list(df['STATUS'].unique()))
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = sorted(list(df['TIPO'].unique()))
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = sorted(list(df['ORIGEM'].unique()))
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = sorted(list(df['TIPOCAUSA'].unique()))
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global_oem
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global_oem = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)
    df_filtered_global_oem
    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(2022, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 5, 9), step=1)

In [5]:
caminho_arquivo_excel = df_filtered_global_oem

In [6]:
df_filtro = caminho_arquivo_excel[['NUMOS', 'UC','IDSIGFI','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]

## DataFrame a partir do arquivo Excel

In [7]:
df_excel =df_filtro[['NUMOS']]
df_excel.count()

NUMOS    137
dtype: int64

# Passo 3: Criar um DataFrame com a lista de arquivos PDF

In [8]:
df_pdf = lista_arquivos_pdf.astype('int64')


#  Passo 4: Comparar os DataFrames para identificar arquivos ausentes no PDF e arquivos não listados no Excel

In [9]:
arquivos_ausentes_no_pdf = df_excel[~df_excel['NUMOS'].isin(df_pdf['NUMOS'])]
arquivos_nao_listados_no_excel = df_pdf[~df_pdf['NUMOS'].isin(df_excel['NUMOS'])]

## Resultados

In [10]:
nao_encontado_pdf = arquivos_ausentes_no_pdf.merge(df_filtro, left_on='NUMOS', right_on='NUMOS', how='left')

display(nao_encontado_pdf)
print(nao_encontado_pdf.shape)

,NUMOS,UC,IDSIGFI,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
0,202400029577,NaN,18149,RAIMUNDO PEREIRA DA SILVA,53.0,VALIDADO,ONEL007,2024-04-27 09:48:00
1,202400028207,NaN,19821,ALE ANUTE SILVA,53.0,VALIDADO,ONEL007,2024-05-05 14:10:00
2,202400028199,NaN,19827,DIENIFA SOUZA DO NASCIMENTO,53.0,VALIDADO,ONEL007,2024-05-03 15:27:00
3,202400028181,NaN,19825,KARINE DE SOUZA NASCIMENTO,53.0,VALIDADO,ONEL007,2024-05-03 14:18:00
4,202400028173,NaN,18216,IGREJA ASSEMBLÉIA DE DEUS SAMAR,53.0,VALIDADO,ONEL007,2024-05-03 11:12:00
...,...,...,...,...,...,...,...,...
68,202400017499,NaN,18813,ROSANGELA SOARES DE LIMA,54.0,VALIDADO,ONEOM003,2024-03-30 13:21:00
69,202400017481,NaN,18419,RAIMUNDO PAULINO FERREIRA,54.0,VALIDADO,ONEOM003,2024-03-31 04:12:00
70,202400017465,NaN,18891,FABRICIO PAIVA FURTADO,54.0,VALIDADO,ONEOM003,2024-03-27 11:06:00
71,202400017457,NaN,18446,ANTONIO LIMA MORAIS,54.0,VALIDADO,ONEOM003,2024-03-27 13:17:00


(73, 8)


In [11]:
nao_encontado_lista = arquivos_nao_listados_no_excel.merge(df_filtro, left_on='NUMOS', right_on='NUMOS', how='left')

display(nao_encontado_lista)
print(nao_encontado_pdf.shape)

,NUMOS,UC,IDSIGFI,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO


(73, 8)


In [12]:
print("listar arquivo duplicados:")
display(lista_arquivos_pdf_duplicada)

print("Arquivos ausentes na lista de PDF:")
display(arquivos_ausentes_no_pdf.shape)

print("\nArquivos não listados no Excel:")
display(arquivos_nao_listados_no_excel.shape)

listar arquivo duplicados:


0

Arquivos ausentes na lista de PDF:


(73, 1)


Arquivos não listados no Excel:


(0, 1)

In [13]:
# Salvando a planilha( .xlsx)
dataframe_excel = pd.ExcelWriter(r"C:\Users\helym\projetos_python\jupyter\INS\output\2ªtranche_preventiva_csv_ausente_hoppecke.xlsx")
nao_encontado_pdf.to_excel(dataframe_excel, index=False)
dataframe_excel.close()